# import packages

In [1]:
from twoboxTask.twobox_IRC_torch import *

In [2]:
# import numpy as np
# #import numpy.ma as ma
import scipy.io as sio
# import matplotlib.pyplot as plt

# from sklearn.decomposition import PCA
# from sklearn import random_projection

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

# import data

In [4]:
data = sio.loadmat('NeuralDatafromNeda/behav_valid.mat')
sch = data['sch']
binsize = 200

In [5]:
"""
index of sessions without NaN values
"""
idx_nonan = list(set([i for i in range(len(data['bLocY'][0])) if len(data['bLocY'][0][i])!= 0]) & 
     set([i for i in range(len(data['bLocX'][0])) if len(data['bLocX'][0][i])!= 0]) & 
     set([i for i in range(len(data['b1PushedTimes'][0])) if len(data['b1PushedTimes'][0][i])!= 0]) & 
     set([i for i in range(len(data['b2PushedTimes'][0])) if len(data['b2PushedTimes'][0][i])!= 0]) &
     set([i for i in range(len(data['rew1DelTimes'][0])) if len(data['rew1DelTimes'][0][i])!= 0]) & 
     set([i for i in range(len(data['rew2DelTimes'][0])) if len(data['rew2DelTimes'][0][i])!= 0]) &
     set([i for i in range(len(data['bLocX'][0])) if data['bLocX'][0][i].shape[1]!= 0])
    )

(np.array(idx_nonan)+1).T

array([ 15,  20,  31,  32,  33,  34,  35,  36,  38,  39,  40,  41,  42,
        45,  46,  63,  64,  67,  68,  69,  71,  74,  76,  79,  80,  84,
        88,  89,  90,  95,  97,  98,  99, 109, 110, 111, 112])

In [6]:
"""
scheudles of the sessions without NaN values
"""
[(ind + 1, sch.squeeze()[ind]) for ind in idx_nonan]

[(15, array([[20, 40, 93]], dtype=uint8)),
 (20, array([[ 20,  40, 100]], dtype=uint8)),
 (31, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (32, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (33, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (34, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (35, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (36, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (38, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (39, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (40, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (41, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (42, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (45, array([[ 20,  20, 100]], dtype=uint8)),
 (46, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (63, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (64, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (67, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (68, array([[15, 25, 34, 25, 15, 66]], 

In [7]:
"""
check which schedule appears most
"""
schs = []
for i in range(len(sch.squeeze()[idx_nonan])):
    sch_i = sch.squeeze()[idx_nonan][i].squeeze()
    if len(sch_i) == 3 and not np.isnan(sch_i[0]) and not np.isnan(sch_i[1]):
        schs.append(sch_i)
    if len(sch_i) == 6 and not np.isnan(sch_i[3]) and not np.isnan(sch_i[4]):
        schs.append(sch_i[:3])
        schs.append(sch_i[3:6])
    elif len(sch_i) == 9 and not np.isnan(sch_i[6]) and not np.isnan(sch_i[7]):
        schs.append(sch_i[:3])
        schs.append(sch_i[3:6])
        schs.append(sch_i[6:9])
    elif len(sch_i) == 12 and not np.isnan(sch_i[9]) and not np.isnan(sch_i[10]):    
        schs.append(sch_i[:3])
        schs.append(sch_i[3:6])
        schs.append(sch_i[6:9])
        schs.append(sch_i[9:12])
        
keys = set([(s[0], s[1]) for s in schs])
[(key, sum([s[2] for s in schs if np.all(s[:2]==np.array(key))])) for key in keys]

[((30, 30), 200),
 ((20, 40), 395),
 ((10, 30), 1108),
 ((30, 10), 234),
 ((20, 20), 100),
 ((25, 15), 1700),
 ((15, 25), 1700)]

In [8]:
target_schedule = np.array((15,25))

# look into the data with selected schedule

In [9]:
"""
index of sessions with a particulat schedule
"""
target_schedule = np.array((15,25))

idx_target = [ind for ind in idx_nonan
       if len(sch.squeeze()[ind][0]) ==3 and np.all(sch.squeeze()[ind][0, 0:2] == target_schedule)
       or len(sch.squeeze()[ind][0]) ==6 and (np.all(sch.squeeze()[ind][0, 0:2] == target_schedule) 
                                              or  np.all(sch.squeeze()[ind][0, 3:5] == target_schedule))
       or len(sch.squeeze()[ind][0]) ==9 and (np.all(sch.squeeze()[ind][0, 0:2] == target_schedule) 
                                              or np.all(sch.squeeze()[ind][0, 3:5] == target_schedule)
                                              or np.all(sch.squeeze()[ind][0, 6:8] == target_schedule))
       or len(sch.squeeze()[ind][0]) ==12 and (np.all(sch.squeeze()[ind][0, 0:2] == target_schedule) 
                                              or np.all(sch.squeeze()[ind][0, 3:5] == target_schedule)
                                              or np.all(sch.squeeze()[ind][0, 6:8] == target_schedule)
                                              or np.all(sch.squeeze()[ind][0, 9:11] == target_schedule))]

[(ind + 1, sch.squeeze()[ind]) for ind in idx_target]

[(31, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (32, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (33, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (34, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (35, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (36, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (38, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (39, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (40, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (41, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (42, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (46, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (63, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (64, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (67, array([[25, 15, 34, 15, 25, 66]], dtype=uint8)),
 (68, array([[15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (74, array([[25, 15, 34, 15, 25, 66, 15, 25, 34, 25, 15, 66]], dtype=uint8)),
 (76, array([[15, 25, 34, 25, 15, 66, 15,

In [10]:
np.array(idx_target).T

array([ 30,  31,  32,  33,  34,  35,  37,  38,  39,  40,  41,  45,  62,
        63,  66,  67,  73,  75,  78,  79,  83, 108, 109, 110, 111])

In [11]:
print("The desired schedule is:", target_schedule)
print("The session index with desired schedule: \n", np.array(idx_target).T)

The desired schedule is: [15 25]
The session index with desired schedule: 
 [ 30  31  32  33  34  35  37  38  39  40  41  45  62  63  66  67  73  75
  78  79  83 108 109 110 111]


# Look into a particular session (ex: 74th sesseion in matlab data)

In [12]:
sess_use_matlab = 74

sess_use = sess_use_matlab - 1
b1PushedTimes = data['b1PushedTimes'][0][sess_use][0]
b2PushedTimes = data['b2PushedTimes'][0][sess_use][0]

rew1DelTimes = data['rew1DelTimes'][0][sess_use][0]
rew2DelTimes = data['rew2DelTimes'][0][sess_use][0]

bLocX = data['bLocX'][0][sess_use][0].astype(float)
bLocY = data['bLocY'][0][sess_use][0].astype(float)


In [13]:
"""
eliminate the bins at the end without location information
"""
b1PushedBins = np.rint(b1PushedTimes/binsize).astype(int)
b2PushedBins = np.rint(b2PushedTimes/binsize).astype(int)
rew1DelBins = np.rint(rew1DelTimes/binsize).astype(int)
rew2DelBins = np.rint(rew2DelTimes/binsize).astype(int)

In [20]:
b1PushedBins = b1PushedBins[np.where(b1PushedBins <= len(bLocX))]
b2PushedBins = b2PushedBins[np.where(b2PushedBins <= len(bLocX))]
rew1DelBins = rew1DelBins[np.where(rew1DelBins <= len(bLocX))]
rew2DelBins = rew2DelBins[np.where(rew2DelBins <= len(bLocX))]

In [21]:
bLocX.shape

((65512,), (91,))

In [22]:
rew1DelBins.shape, rew2DelBins.shape

((109,), (91,))

In [23]:
b1PushedBins.shape, b2PushedBins.shape

((296,), (208,))

In [24]:
target_schedule

array([15, 25])

In [ ]:
rewDelBins = mergeArrays(rew1DelBins, rew2DelBins)
rewDel_subsess = [rewDelBins[0:34], rewDelBins[34:100],rewDelBins[100:134],rewDelBins[134:]]

In [ ]:
# time of push and del for each of the sechdules

b1PushedBins_subsess = [[], [], [], []]
b2PushedBins_subsess = [[], [], [], []]

b1PushedBins_subsess_del = [[], [], [], []]
b2PushedBins_subsess_del = [[], [], [], []]

In [ ]:
ind = 0
for i in range(len(rewDel_subsess)):
    start = rewDel_subsess[i][0]   
    end = rewDel_subsess[i][-1]
    
    #print(b1PushedBins[ind], start, end)
    while ind < len(b1PushedBins) and b1PushedBins[ind]<= end:
        #print('add', b1PushedBins[ind])
        b1PushedBins_subsess[i].append(b1PushedBins[ind])
        
        if (b1PushedBins[ind] in rewDel_subsess[i] or b1PushedBins[ind] + 1 in rewDel_subsess[i]
            or b1PushedBins[ind] + 2 in rewDel_subsess[i]
            or b1PushedBins[ind] + 3 in rewDel_subsess[i]
            or b1PushedBins[ind] + 4 in rewDel_subsess[i]):
            b1PushedBins_subsess_del[i].append(b1PushedBins[ind])
        
        ind += 1
          

In [ ]:
ind = 0
for i in range(len(rewDel_subsess)):
    start = rewDel_subsess[i][0]
    end = rewDel_subsess[i][-1]
    
    #print(b1PushedBins[ind], start, end)
    while ind < len(b2PushedBins) and b2PushedBins[ind]<= end:
        b2PushedBins_subsess[i].append(b2PushedBins[ind])
        
        if (b2PushedBins[ind] in rewDel_subsess[i] 
            or b2PushedBins[ind] + 1 in rewDel_subsess[i] 
            or b2PushedBins[ind] + 2 in rewDel_subsess[i]
            or b2PushedBins[ind] + 3 in rewDel_subsess[i]
            or b2PushedBins[ind] + 4 in rewDel_subsess[i]):
            b2PushedBins_subsess_del[i].append(b2PushedBins[ind])
            
        ind += 1
          

In [ ]:
print('Schedule of sub-session in the {}-th trial is: {}'.format(sess_use_matlab, sch.squeeze()[sess_use_matlab - 1]))

print('Number of button pressing on box 1 in each session: {}'.format([len(i) for i in b1PushedBins_subsess]))
print('Number of button pressing on box 2 in each session: {}'.format([len(i) for i in b2PushedBins_subsess]))

print('Number of successful button pressing on box 1 in each session: {}'.format([len(i) for i in b1PushedBins_subsess_del]))
print('Number of successful button pressing on box 2 in each session: {}'.format([len(i) for i in b2PushedBins_subsess_del]))




In [ ]:
#pb_time = np.array(b1PushedBins_subsess[1])

In [ ]:
target_schedule

In [ ]:
"""
plot the locations and pb times
"""
b1Pushed = np.zeros(len(bLocX))
b1Pushed[b1PushedBins] = 500
b2Pushed = np.zeros(len(bLocX))
b2Pushed[b2PushedBins] = 300
plt.plot(bLocX[:])
plt.plot(b1Pushed[:], 'r.')
plt.plot(b2Pushed[:], 'm.')
plt.plot(np.linspace(0, 65000), np.ones(np.linspace(0, 65000).shape) * np.nanmean(bLocX[list(b1PushedBins)]))
plt.plot(np.linspace(0, 65000), np.ones(np.linspace(0, 65000).shape) * (np.nanmean(bLocX[list(b1PushedBins)])
                                                                        + 3 *np.nanstd(bLocX[list(b1PushedBins)])))
plt.plot(np.linspace(0, 65000), np.ones(np.linspace(0, 65000).shape) * (np.nanmean(bLocX[list(b1PushedBins)])
                                                                        - 3 *np.nanstd(bLocX[list(b1PushedBins)])))
plt.show()

In [ ]:
# bin_idx = np.concatenate((np.arange(bin_idx_range[0][0], bin_idx_range[0][1]), 
#                           np.arange(bin_idx_range[1][0], bin_idx_range[1][1])))

# #bin_idx = np.arange(bin_idx_range[0][0], bin_idx_range[0][1])
# bin_idx

In [ ]:
"""
BOX1 locations based on actions, filter out the values that are nan or outliers
"""
box1push = bLocX[b1PushedBins]
mask1  = ~np.isnan(box1push)  # the elements that are not nan
mask1[mask1] &= (box1push[mask1] > (np.nanmean(box1push) - 3 * np.nanstd(box1push)))  # elements that are not outlier
invalid_idx1 = np.where(mask1 == False)[0] 
bLocX[b1PushedBins[invalid_idx1]] = np.nan # mark the elements that are either nan or outlier as "nan"

"""
BOX2 locations, filter out the values that are nan or outliers
"""
box2push = bLocX[b2PushedBins]
mask2  = ~np.isnan(box2push)  # the elements that are not nan
mask2[mask2] &= (box2push[mask2] > (np.nanmean(box2push) - 3 * np.nanstd(box2push)))  # elements that are not outlier
invalid_idx2 = np.where(mask2 == False)[0]
bLocX[b2PushedBins[invalid_idx2]] = np.nan # mark the elements that are either nan or outlier as "nan"


In [ ]:
"""
nan elements are filled by a smoothing filter that takes average of the previous three elements 
"""
win_wid = 3
bLocX_temp = np.concatenate((np.ones(win_wid) * np.nanmean(bLocX), bLocX))
for i in np.where(np.isnan(bLocX) == True)[0]:
    bLocX_temp[i+win_wid] = np.mean(bLocX_temp[i:i+win_wid])
    
bLocX = bLocX_temp[win_wid:]

In [ ]:
"""
The boundaries to descretize the locations are decided by the 
fartherest location when button push happens
"""
box1_boud = min(bLocX[b1PushedBins] )
box2_boud = max(bLocX[b2PushedBins] )
box1_boud, box2_boud

In [ ]:
plt.plot(bLocX[:], bLocY[:], 'b.')
plt.plot(bLocX[b1PushedBins], bLocY[b1PushedBins], 'r.')
plt.plot(bLocX[b2PushedBins], bLocY[b2PushedBins], 'g.')
plt.plot(box1_boud * np.ones(np.linspace(0, 400).shape), np.linspace(0, 400))
plt.plot(box2_boud * np.ones(np.linspace(0, 400).shape), np.linspace(0, 400))
plt.show()

In [ ]:
"""
discretize location as center(loc = 0), box1 (loc = 1) and box2(loc = 2)

"""
locT = np.zeros(np.shape(bLocX), dtype = int)        # if at center location, loc = 0
locT[np.where(bLocX < box2_boud)] = 2   # at box2 (left side)
locT[np.where(bLocX > box1_boud)] = 1   # at box1 (right side)

In [ ]:
"""
actions
"""

a0 = 0    # a0 = do nothing
g0 = 1    # g0 = go to location 0
g1 = 2    # g1 = go toward box 1 (via location 0 if from 2)
g2 = 3    # g2 = go toward box 2 (via location 0 if from 1
pb = 4    # pb  = push button

actT = np.zeros(np.shape(bLocX), dtype = int) 
actT[b1PushedBins] = pb
actT[b2PushedBins] = pb
#actT[rew1DelBins - 1] = pb
#actT[rew2DelBins-1] = pb

for t in np.where(locT == 0)[0]:
    if t == len(bLocX) - 1:
        break
    if locT[t+1] == 1:
        actT[t] = g1
    elif locT[t+1] == 2:
        actT[t] = g2

for t in np.where(locT == 1)[0]:
    if t == len(bLocX) - 1:
        break
    if locT[t+1] != 1:
        actT[t] = g2
        
for t in np.where(locT == 2)[0]:
    if t == len(bLocX) - 1:
        break
    if locT[t+1] != 2:
        actT[t] = g1

In [ ]:
# rew1DelBins_modified = []
# for i in b1PushedBins:
#     if i in rew1DelBins or i + 1 in rew1DelBins or i+2 in rew1DelBins or i+3 in rew1DelBins or i+4 in rew1DelBins or i+5 in rew1DelBins:
#         rew1DelBins_modified.append(i+1)
        
# rew2DelBins_modified = []
# for i in b2PushedBins:
#     if i in rew2DelBins or i + 1 in rew2DelBins or i+2 in rew2DelBins or i+3 in rew2DelBins or i+4 in rew2DelBins or i+5 in rew2DelBins:
#         rew2DelBins_modified.append(i+1)

In [ ]:
"""
rewards
"""
rewT = np.zeros(np.shape(bLocX), dtype = int) 
rewT[rew1DelBins] = 1
rewT[rew2DelBins] = 1

In [ ]:
"""
combine data with the same schedule from one session, 
the list components indicate the starting and ending time of a timeframe,
eliminate the first 10 trials after schedule change
"""
bin_idx_range = []  # bin range of reward delivery
removeFirst = 10
start = 0
trialBins = mergeArrays(rew1DelBins, rew2DelBins)
prev_sess_target = 0   # default is the previous is selected, no need to remove

if len(sch.squeeze()[sess_use][0]) >=3:
    if np.all(sch.squeeze()[sess_use][0, 0:2] == target_schedule):
        valid_DelBins = trialBins[range(start + removeFirst * (1 - prev_sess_target), start + sch.squeeze()[sess_use][0, 2])] 
        bin_idx_range.append([valid_DelBins[0],valid_DelBins[-1]]) 
        prev_sess_target = 1
    else:
        prev_sess_target = 0
    start += sch.squeeze()[sess_use][0, 2]    

if len(sch.squeeze()[sess_use][0]) >=6:
    if np.all(sch.squeeze()[sess_use][0, 3:5] == target_schedule):
        valid_DelBins = trialBins[range(start + removeFirst * (1 - prev_sess_target), start + sch.squeeze()[sess_use][0, 5])] 
        bin_idx_range.append([valid_DelBins[0],valid_DelBins[-1]]) 
        prev_sess_target = 1
    else:
        prev_sess_target = 0
    start += sch.squeeze()[sess_use][0, 5]


if len(sch.squeeze()[sess_use][0]) >= 9:
    if np.all(sch.squeeze()[sess_use][0, 6:8] == target_schedule):
        valid_DelBins = trialBins[range(start + removeFirst * (1 - prev_sess_target), start + sch.squeeze()[sess_use][0, 8])] 
        bin_idx_range.append([valid_DelBins[0],valid_DelBins[-1]]) 
        prev_sess_target = 1
    else:
        prev_sess_target = 0
    start += sch.squeeze()[sess_use][0, 8]
    
if len(sch.squeeze()[sess_use][0]) >= 12:
    if np.all(sch.squeeze()[sess_use][0, 9:11] == target_schedule):
        valid_DelBins = trialBins[range(start + removeFirst * (1 - prev_sess_target), start + sch.squeeze()[sess_use][0, 11])] 
        bin_idx_range.append([valid_DelBins[0],valid_DelBins[-1]]) 
        prev_sess_target = 1
    else:
        prev_sess_target = 0
    start += sch.squeeze()[sess_use][0, 11]

bin_idx_range

In [ ]:
# """
# combine data with the same schedule from one session, 
# the list components indicate the starting and ending time of a timeframe,
# eliminate the first 10 trials after schedule change
# """
# bin_idx_range = []  # bin range of reward delivery
# removeFirst = 10
# start = 0
# trialBins = mergeArrays(rew1DelBins, rew2DelBins)

# if len(sch.squeeze()[sess_use][0]) >=3:
#     if np.all(sch.squeeze()[sess_use][0, 0:2] == target_schedule):
#         valid_DelBins = trialBins[range(start + removeFirst, start + sch.squeeze()[sess_use][0, 2])] 
#         bin_idx_range.append([valid_DelBins[0],valid_DelBins[-1]]) 
#         #print(valid_DelBins)
#     start += sch.squeeze()[sess_use][0, 2]
        

# if len(sch.squeeze()[sess_use][0]) >=6:
#     if np.all(sch.squeeze()[sess_use][0, 3:5] == target_schedule):
#         valid_DelBins = trialBins[range(start + removeFirst, start + sch.squeeze()[sess_use][0, 5])] 
#         bin_idx_range.append([valid_DelBins[0],valid_DelBins[-1]]) 
#         #print(valid_DelBins)
#     start += sch.squeeze()[sess_use][0, 5]


# if len(sch.squeeze()[sess_use][0]) >= 9:
#     if np.all(sch.squeeze()[sess_use][0, 6:8] == target_schedule):
#         valid_DelBins = trialBins[range(start + removeFirst, start + sch.squeeze()[sess_use][0, 8])] 
#         bin_idx_range.append([valid_DelBins[0],valid_DelBins[-1]]) 
#         #print(valid_DelBins)
#     start += sch.squeeze()[sess_use][0, 8]
    
# if len(sch.squeeze()[sess_use][0]) >= 12:
#     if np.all(sch.squeeze()[sess_use][0, 9:11] == target_schedule):
#         valid_DelBins = trialBins[range(start + removeFirst, start + sch.squeeze()[sess_use][0, 11])] 
#         bin_idx_range.append([valid_DelBins[0],valid_DelBins[-1]]) 
#         #print(valid_DelBins)
#     start += sch.squeeze()[sess_use][0, 11]

# bin_idx_range

In [ ]:
lower1 = bin_idx_range[0][0]
upper1 = bin_idx_range[0][1]
bin_idx1 = np.arange(lower1, upper1)

fig1, (ax1, ax2) = plt.subplots(2,1, figsize = (18,9))
ax1.plot(bin_idx1, bLocX[bin_idx1], 'b')
ax1.plot(b1PushedBins[(b1PushedBins >= lower1) & (b1PushedBins <= upper1)],
         bLocX[b1PushedBins[(b1PushedBins >= lower1) & (b1PushedBins <= upper1)]], 'r.')
ax1.plot(b2PushedBins[(b2PushedBins >= lower1) & (b2PushedBins <= upper1)],
         bLocX[b2PushedBins[(b2PushedBins >= lower1) & (b2PushedBins <= upper1)]], 'm.')
ax1.plot(rew1DelBins[(rew1DelBins >= lower1) & (rew1DelBins <= upper1)],
         bLocX[rew1DelBins[(rew1DelBins >= lower1) & (rew1DelBins <= upper1)]], 'g*')
ax1.plot(rew2DelBins[(rew2DelBins >= lower1) & (rew2DelBins <= upper1)],
         bLocX[rew2DelBins[(rew2DelBins >= lower1) & (rew2DelBins <= upper1)]], 'c*')
#ax1.plot(bin_idx1, locT[bin_idx1] * 200, 'g*')
ax1.plot(np.linspace(lower1, upper1), np.ones(np.linspace(lower1, upper1).shape) * box1_boud)
ax1.plot(np.linspace(lower1, upper1), np.ones(np.linspace(lower1, upper1).shape) * box2_boud)
ax1.yaxis.tick_right()
ax1.set_yticks([box2_boud, box1_boud])
labels = [item.get_text() for item in ax1.get_yticklabels()]
labels[1] = 'box1'
labels[0] = 'box2'
ax1.set_yticklabels(labels)
ax1.set_title('behavior in the 1st sub-session')
ax1.set_ylabel('location (along the cage)')

lower2 = bin_idx_range[1][0]
upper2 = bin_idx_range[1][1]
bin_idx2 = np.arange(lower2, upper2)

ax2.plot(bin_idx2, bLocX[bin_idx2], 'b', label = 'location' )
ax2.plot(b1PushedBins[(b1PushedBins >= lower2) & (b1PushedBins <= upper2)],
         bLocX[b1PushedBins[(b1PushedBins >= lower2) & (b1PushedBins <= upper2)]], 'r.', label = 'open box 1')
ax2.plot(b2PushedBins[(b2PushedBins >= lower2) & (b2PushedBins <= upper2)],
         bLocX[b2PushedBins[(b2PushedBins >= lower2) & (b2PushedBins <= upper2)]], 'm.', label = 'open box 2')
ax2.plot(rew1DelBins[(rew1DelBins >= lower2) & (rew1DelBins <= upper2)],
         bLocX[rew1DelBins[(rew1DelBins >= lower2) & (rew1DelBins <= upper2)]], 'g*')
ax2.plot(rew2DelBins[(rew2DelBins >= lower2) & (rew2DelBins <= upper2)],
         bLocX[rew2DelBins[(rew2DelBins >= lower2) & (rew2DelBins <= upper2)]], 'c*')
ax2.plot(np.linspace(lower2, upper2), np.ones(np.linspace(lower2, upper2).shape) * box1_boud)
ax2.plot(np.linspace(lower2, upper2), np.ones(np.linspace(lower2, upper2).shape) * box2_boud)
ax2.yaxis.tick_right()
ax2.set_yticks([box2_boud, box1_boud])
labels = [item.get_text() for item in ax2.get_yticklabels()]
labels[1] = 'box1'
labels[0] = 'box2'
ax2.set_yticklabels(labels)
ax2.legend(loc = 'upper right')

ax2.set_title('behavior in the 2nd sub-session')
ax2.set_ylabel('location (along the cage)')
ax2.set_xlabel('time')
plt.show()

In [ ]:
# """
# save pre-processed data to file
# """
# path = os.getcwd()
# data_dict = {'actions': actT,
#              'rewards': rewT,
#              'locations': locT,
#             'bin_idx_range': bin_idx_range,
#             'target_schedule': target_schedule}
# data_output = open(path + '/twoboxTask/Data/monkey_twobox_preprocessed.pkl', 'wb')
# pickle.dump(data_dict, data_output)
# data_output.close()

In [ ]:
"""
stack inforamtion from one session
combine differnet time frames, padded with zero inbetween
"""
# actTT = np.hstack((actT[range(bin_idx_range[0][0], bin_idx_range[0][1]+1)], 0, 
#                               actT[range(bin_idx_range[1][0], bin_idx_range[1][1]+1)]))
# rewTT = np.hstack((rewT[range(bin_idx_range[0][0], bin_idx_range[0][1]+1)], 0, 
#                              rewT[range(bin_idx_range[1][0], bin_idx_range[1][1]+1)]))
# locTT = np.hstack((locT[range(bin_idx_range[0][0], bin_idx_range[0][1]+1)], 0, 
#                              locT[range(bin_idx_range[1][0], bin_idx_range[1][1]+1)]))

# actTT = np.hstack((actT[bin_idx2]))
# rewTT = np.hstack((rewT[bin_idx2]))
# locTT = np.hstack((locT[bin_idx2]))

"""
consider subsessions where the animals incolce actively
"""
# actTT = np.hstack((actT[range(17000, bin_idx_range[0][1])]))
# rewTT = np.hstack((rewT[range(17000, bin_idx_range[0][1])]))
# locTT = np.hstack((locT[range(17000, bin_idx_range[0][1])]))

actTT = np.hstack((actT[range(bin_idx_range[1][0], 48000)]))
rewTT = np.hstack((rewT[range(bin_idx_range[1][0], 48000)]))
locTT = np.hstack((locT[range(bin_idx_range[1][0], 48000)]))

obsTT = (np.vstack([actTT, rewTT, locTT]).T)

In [ ]:
T = obsTT.shape[0]

obs = obsTT[:T, :]


act = obs[:, 0]
rew = obs[:, 1]
loc = obs[:, 2]

In [ ]:
showlen = T
showT = np.arange(showlen)
fig_posterior, ax_loc = plt.subplots(1, 1, figsize= (10, 4))

ax_loc.plot((np.remainder(loc[showT]+1, 3) - 1 ) * 10, 'g.-', markersize = 8, linewidth = 3)
box1_r = act[showT] // 4 * 7 * np.remainder(loc[showT]+1, 3) * np.insert(rew[showT][1:], -1, 0) * 1.0 
box2_r = act[showT] // 4 * 7 * (np.remainder(loc[showT]+1, 3) - 2) * np.insert(rew[showT][1:], -1, 0) * 1.0
box1_n = act[showT] // 4 * 7 * np.remainder(loc[showT]+1, 3) * (1-np.insert(rew[showT][1:], -1, 0))* 1.0 * (loc[showT] != 0)
box2_n = act[showT] // 4 * 7 * ((np.remainder(loc[showT]+1, 3) - 2) * (1-np.insert(rew[showT][1:], -1, 0)))* 1.0 * (loc[showT] != 0)
box1_r[ box1_r==0 ] = np.nan
box2_n[ box2_n==0 ] = np.nan
box2_r[ box2_r==0 ] = np.nan
box1_n[ box1_n==0 ] = np.nan
ax_loc.plot(box2_r, '^', c = 'red', markersize = 15)
ax_loc.plot(box1_n, 'v', c = 'blue', markersize = 15)
ax_loc.plot(box2_n, '^', c = 'blue', markersize = 15)
ax_loc.plot(box1_r, 'v', c = 'red', markersize = 15)

ax_loc.set_xlim([0,showlen])
ax_loc.spines['top'].set_visible(False)
ax_loc.spines['right'].set_visible(False)
ax_loc.spines['bottom'].set_visible(False)
ax_loc.spines['left'].set_visible(False)
#ax_loc.set_ylim([1,16])
ax_loc.set_yticks([])
#ax_loc.set_xticks([])



plt.tight_layout()
plt.show()



In [ ]:
# print(temp)

# IRC

In [ ]:
obsN = np.expand_dims(obs, axis=0)

In [ ]:
# beta = 0                         # available food dropped back into box after button press
# gamma1 = self.parameters[0]      # reward becomes available in box 1
# gamma2 = self.parameters[1]      # reward becomes available in box 2
# delta = 0                        # animal trips, doesn't go to target location
# direct = self.parameters[7]      # animal goes right to target, skipping location 0
# epsilon1 = 0 #self.parameters[2] # available food disappears from box 1
# epsilon2 = 0 #self.parameters[3] # available food disappears from box 2
# rho = 1                          # food in mouth is consumed
# # State rewards
# Reward = 1                       # reward per time step with food in mouth
# Groom = self.parameters[2]       # location 0 reward

# # Action costs
# travelCost = self.parameters[3]
# pushButtonCost = self.parameters[4]
# sigmaTb = self.parameters[6]

In [ ]:
nq = 10
nr = 2
nl = 3
na = 5
discount = 0.99

In [ ]:
sample_length = len(obs)
sample_number = 1

app_rate1_ini = 0.0577
disapp_rate1_ini = .0232
app_rate2_ini = 0.0859
disapp_rate2_ini = .0182
food_missed_ini = .1193  # 0
food_consumed_ini = .9331  # 1
belief_diffusion_ini = .0380  # 0.03827322 # .1
policy_temperature_ini = .2034  # 0.15768841  # .06
push_button_cost_ini = .3244  # 0.40363519  # .2
grooming_reward_ini = .0130  # 0.20094441  # .3
travel_cost_ini = .3951  # 0.3033027  # .1
trip_prob_ini = .0453
direct_prob_ini = 0.1954  # .1

app_rate1_ini = torch.autograd.Variable(torch.tensor([app_rate1_ini]), requires_grad=True)
disapp_rate1_ini = torch.autograd.Variable(torch.tensor([disapp_rate1_ini]), requires_grad=True)
app_rate2_ini = torch.autograd.Variable(torch.tensor([app_rate2_ini]), requires_grad=True)
disapp_rate2_ini = torch.autograd.Variable(torch.tensor([disapp_rate2_ini]), requires_grad=True)
food_missed_ini = torch.autograd.Variable(torch.tensor([food_missed_ini]), requires_grad=True)  # 0
food_consumed_ini = torch.autograd.Variable(torch.tensor([food_consumed_ini]), requires_grad=True)  # .99 #1
belief_diffusion_ini = torch.autograd.Variable(torch.tensor([belief_diffusion_ini]), requires_grad=True)  # .1
policy_temperature_ini = torch.autograd.Variable(torch.tensor([policy_temperature_ini]), requires_grad=True)  # .061
push_button_cost_ini = torch.autograd.Variable(torch.tensor([push_button_cost_ini]), requires_grad=True)  # .3
grooming_reward_ini = torch.autograd.Variable(torch.tensor([grooming_reward_ini]), requires_grad=True)  # .3
travel_cost_ini = torch.autograd.Variable(torch.tensor([travel_cost_ini]), requires_grad=True)  # .3
direct_prob_ini = torch.autograd.Variable(torch.tensor([direct_prob_ini]), requires_grad=True)  # .3
trip_prob_ini = torch.autograd.Variable(torch.tensor([trip_prob_ini]), requires_grad=True)  # .3

point_ini = collections.OrderedDict()
point_ini['food_missed'] = food_missed_ini
point_ini['app_rate1'] = app_rate1_ini
point_ini['disapp_rate1'] = disapp_rate1_ini
point_ini['app_rate2'] = app_rate2_ini
point_ini['disapp_rate2'] = disapp_rate2_ini
point_ini['food_consumed'] = food_consumed_ini
point_ini['push_button_cost'] = push_button_cost_ini
point_ini['belief_diffusion'] = belief_diffusion_ini
point_ini['policy_temperature'] = policy_temperature_ini
point_ini['direct_prob'] = direct_prob_ini
point_ini['trip_prob'] = trip_prob_ini
point_ini['grooming_reward'] = grooming_reward_ini
point_ini['travel_cost'] = travel_cost_ini

In [ ]:
LR = 10**-6*1
EPS = 0.1
BATCH_SIZE = 1

In [ ]:
IRC_monkey = twobox_IRC_torch(discount, nq, nr, na, nl, point_ini)

In [ ]:
IRC_monkey.IRC_batch(obsN[:, :1000, :], lr=LR, eps=EPS, batch_size=BATCH_SIZE, shuffle=True)
print(IRC_monkey.log_likelihood_traj[-1])

In [ ]:
plt.plot(IRC_monkey.log_likelihood_traj)
plt.show()

In [ ]:
print(temp)

# posterior of beliefs

In [ ]:
para_test

In [ ]:
twobox_est = twoboxMDP(discount, nq, nr, na, nl, para_test)
twobox_est.setupMDP()
twobox_est.solveMDP_sfm()
ThA = twobox_est.ThA
softpolicy = twobox_est.softpolicy
pi = np.ones(nq * nq)/ nq /nq  # initialize the estimation of the belief state
twoHMM_est = HMMtwobox(ThA, softpolicy, pi)

In [ ]:
alpha_est, scale_est = twoHMM_est.forward_scale(obs)
beta_est = twoHMM_est.backward_scale(obs, scale_est)
gamma_est = twoHMM_est.compute_gamma(alpha_est, beta_est)
xi_est = twoHMM_est.compute_xi(alpha_est, beta_est, obs)


In [ ]:
belief1_est = np.sum(np.reshape(gamma_est[:, ].T, (gamma_est.shape[1], nq, nq)), axis = 2)
belief2_est = np.sum(np.reshape(gamma_est[:, ].T, (gamma_est.shape[1], nq, nq)), axis = 1)

# belief1_est = np.sum(np.reshape(alpha_est[:, ].T, (alpha_est.shape[1], nq, nq)), axis = 2)
# belief2_est = np.sum(np.reshape(alpha_est[:, ].T, (alpha_est.shape[1], nq, nq)), axis = 1)


In [ ]:
# showlen = 500
# startT = 0
# showT = range(startT,startT+showlen)


In [ ]:
#fig_posterior = plt.figure(figsize= (15, 10))

showlen = T // 1
startT = T // 1 * (1-1)

endT = startT + showlen
showT = range(startT,endT)

fig, [ax1, ax_loc, ax2] = plt.subplots(3, 1, figsize= (10, 6))

ax1.imshow(belief1_est[showT].T, interpolation='Nearest', cmap='gray', origin='lower', aspect='auto')
ax1.yaxis.set_label_coords(-0.1,0.25)
ax1.set_ylabel('Belief posterior \n about box 1', rotation= 360, fontsize = 15)
ax1.set_yticks([0, nq-1])
ax1.set_yticklabels(['0','1'])
ax1.tick_params(axis='both', which='major', labelsize=18)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.set_xlim([0,showlen])

ax_loc.plot((np.remainder(loc[showT]+1, 3) - 1 ) * 10, 'g-o', markersize = 3)
box1_r = act[showT] // 4 * 7 * np.remainder(loc[showT]+1, 3) * np.insert(rew[showT][1:], -1, 0) * 1.0
box2_r = act[showT] // 4 * 7 * (np.remainder(loc[showT]+1, 3) - 2) * np.insert(rew[showT][1:], -1, 0) * 1.0
box1_n = act[showT] // 4 * 7 * np.remainder(loc[showT]+1, 3) * (1-np.insert(rew[showT][1:], -1, 0))* 1.0
box2_n = act[showT] // 4 * 7 * ((np.remainder(loc[showT]+1, 3) - 2) * (1-np.insert(rew[showT][1:], -1, 0)))* 1.0
box1_r[ box1_r==0 ] = np.nan
box2_n[ box2_n==0 ] = np.nan
box2_r[ box2_r==0 ] = np.nan
box1_n[ box1_n==0 ] = np.nan
ax_loc.plot(box2_r, '^', c = 'red', markersize = 10)
ax_loc.plot(box1_n, 'v', c = 'blue', markersize = 10)
ax_loc.plot(box2_n, '^', c = 'blue', markersize = 10)
ax_loc.plot(box1_r, 'v', c = 'red', markersize = 10)
ax_loc.set_xlim([0,showlen])
ax_loc.spines['top'].set_visible(False)
ax_loc.spines['right'].set_visible(False)
ax_loc.spines['bottom'].set_visible(False)
ax_loc.spines['left'].set_visible(False)
ax_loc.set_yticks([])

ax2.imshow(belief2_est[showT].T, interpolation='Nearest', cmap='gray', origin='lower',aspect='auto')
ax2.set_xlabel('time', fontsize = 18)
ax2.yaxis.set_label_coords(-0.1,0.25)
ax2.set_ylabel('Belief posterior \n about box 2', rotation= 360, fontsize = 15)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.set_yticks([0, nq-1])
ax2.set_yticklabels(['0','1'])
ax2.set_xlim([0,showlen])


plt.tight_layout()
plt.show()



In [ ]:
belief_vector = (np.arange(nq) + 1/2)/nq
belief_vector

In [ ]:
belief1_postmean = belief1_est.dot(belief_vector)
belief2_postmean = belief2_est.dot(belief_vector)

belief1_est_MAP = (np.argmax(belief1_est, axis = 1) + 0.5)/nq
belief2_est_MAP = (np.argmax(belief2_est, axis = 1) + 0.5)/nq

In [ ]:
plt.plot(belief1_postmean)
plt.show()

In [ ]:
startSess = lower2  # The start of blocks of trials with target schedule in a session
trueProb1 = np.zeros(upper2 - startSess)  # true probabilities of box 1, for the whole block of trials with target schedule
trueProb2 = np.zeros(upper2 - startSess)

trueProb1_agent = np.zeros(upper2 - startSess)  # true probabilities of box 1, for the whole block of trials with target schedule
trueProb2_agent = np.zeros(upper2 - startSess)


pressBin1 = b1PushedBins[np.where((b1PushedBins >= lower2) & (b1PushedBins <= upper2))]
pressBin2 = b2PushedBins[np.where((b2PushedBins >= lower2) & (b2PushedBins <= upper2))]

trueProb1[0 : pressBin1[0]- startSess] = 1 - np.exp(-1/target_schedule[0] * (200/1000) * range(pressBin1[0]- startSess))
for i in range(len(pressBin1[1:])):
    trueProb1[pressBin1[i - 1]- startSess : pressBin1[i]- startSess] = \
    1 - np.exp(-1/target_schedule[0] * (200/1000) * range(pressBin1[i]- pressBin1[i-1]))

trueProb2[0:pressBin2[0]- startSess] = 1 - np.exp(-1/target_schedule[1] * (200/1000) * range(pressBin2[0]- startSess))
for i in range(len(pressBin2[1:])):
    trueProb2[pressBin2[i - 1]- startSess : pressBin2[i]- startSess] = \
    1 - np.exp(-1/target_schedule[1] * (200/1000) * range(pressBin2[i] - pressBin2[i-1]))


rate0 = (-1/np.log(1 - para_est[0]))/5
rate1 = (-1/np.log(1 - para_est[1]))/5
trueProb1_agent[0 : pressBin1[0]- startSess] = 1 - np.exp(-1/target_schedule[0] * (200/1000) * range(pressBin1[0]- startSess))
for i in range(len(pressBin1[1:])):
    trueProb1_agent[pressBin1[i - 1]- startSess : pressBin1[i]- startSess] = \
    1 - np.exp(-1/rate0 * (200/1000) * range(pressBin1[i]- pressBin1[i-1]))

trueProb2_agent[0:pressBin2[0]- startSess] = 1 - np.exp(-1/target_schedule[1] * (200/1000) * range(pressBin2[0]- startSess))
for i in range(len(pressBin2[1:])):
    trueProb2_agent[pressBin2[i - 1]- startSess : pressBin2[i]- startSess] = \
    1 - np.exp(-1/rate1 * (200/1000) * range(pressBin2[i] - pressBin2[i-1]))



real data, can try on simulated one box task first to understand the dynamic

1. allowing disapperaing rate, would increase likelihood

2. belief temperature 

3. both

4. lapse rate 

In [ ]:
f, axes = plt.subplots(2,2, figsize = (10,10))

axes[0,0].plot(trueProb1[lower2 - startSess : lower2 - startSess + T], 'b', label = 'reward probability (world parameter)')
axes[0,0].plot(belief1_postmean, 'm', label = 'belief posterior mean')
axes[0,0].plot(trueProb1_agent[lower2 - startSess : lower2 - startSess + T], 'r', label = 'reward probability (agent parameter) ')
axes[0,0].set_title('box 1')
axes[0,0].set_xlabel('time')
axes[0,0].set_ylabel('true probability vs mean belief posterior')
legend = axes[0,0].legend(loc='lower center', shadow=True)
#legend.get_frame().set_facecolor('C0')

axes[0,1].scatter(trueProb1[lower2 - startSess : lower2 - startSess + T], belief1_postmean)
axes[0,1].set_title('box 1')
axes[0,1].set_ylabel('mean belief posterior')
axes[0,1].set_xlabel('true probability')

axes[1,0].plot(trueProb2[lower2 - startSess : lower2 - startSess + T], label = 'reward probability (world parameter)')
axes[1,0].plot(belief2_postmean, label = 'belief posterior mean')
axes[1,0].plot(trueProb2_agent[lower2 - startSess : lower2 - startSess + T], label = 'reward probability (agent parameter)')
axes[1,0].set_title('box 2')
axes[1,0].set_xlabel('time')
axes[1,0].set_ylabel('true probability vs mean belief posterior')
axes[1,0].legend(loc='lower center', shadow=True)


axes[1,1].scatter(trueProb2[lower2 - startSess : lower2 - startSess + T], belief2_postmean)
axes[1,1].set_title('box 2')
axes[1,1].set_ylabel('mean belief posterior')
axes[1,1].set_xlabel('true probability')

#f.savefig('belief_rewProb.pdf', format='pdf')

plt.show()


In [ ]:
#np.savetxt('data.csv', np.hstack((belief1_est, belief2_est)), delimiter=',')

In [ ]:
%who

In [ ]:
IRC_monkey

In [ ]:
from datetime import datetime
import os
import pickle

path = os.getcwd()

IRC_dict = {#'LL_slice': LL_slice,
            'log_likelihood_all': log_likelihood_all,
            'point_all': point_all,
            'uValue': IRC_monkey.uValue,
            'vValue': IRC_monkey.vValue,
            'contour_LL_mesh': IRC_monkey.contour_LL_mesh
            }

# # create a file that saves the parameter dictionary using pickle
# para_output = open(path + '/Results/08132019(0014)_data08132019(0013)_agentNN08132019(0026)' + '_IRC0plane'+'_twoboxCol' + '.pkl', 'wb')


para_output = open(path + '/Results/1009_IRC'+ str(2) + '.pkl', 'wb')
pickle.dump(IRC_dict, para_output)
para_output.close()

# Compare IRC results

In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt 
path = os.getcwd()



In [ ]:
"""
import IRC data 1
"""
dataN_pkl_file_IRC1 = open(path + '/Results/1009_IRC'+ str(1) + '.pkl', 'rb')
dataN_pkl_IRC1 = pickle.load(dataN_pkl_file_IRC1)
dataN_pkl_file_IRC1.close()

In [ ]:
"""
import IRC data 2
"""
dataN_pkl_file_IRC2 = open(path + '/Results/1009_IRC'+ str(2) + '.pkl', 'rb')
dataN_pkl_IRC2 = pickle.load(dataN_pkl_file_IRC2)
dataN_pkl_file_IRC2.close()

In [ ]:
point_ini1 = dataN_pkl_IRC1['point_all'][0]
point_est1 = dataN_pkl_IRC1['point_all'][-1]
point_ini2 = dataN_pkl_IRC2['point_all'][0]
point_est2 = dataN_pkl_IRC2['point_all'][-1]

In [ ]:
# point_ini1, point_est1

In [ ]:
# point_ini2, point_est2

In [ ]:
"""
Comparison of parameters
"""

N = 8
ind = np.arange(N)  # the x locations for the groups
width = 0.17       # the width of the bars

fig_parameters = plt.figure(figsize = (12, 5))
ax = fig_parameters.add_subplot(111)

xvals = point_ini1
yvals = point_est1
rects1 = ax.bar(ind, xvals, width, color='r')
rects2 = ax.bar(ind+width, yvals, width = width, color = 'b', 
                align='center', alpha=1, ecolor='black', capsize=8)

zvals = point_ini2
kvals = point_est2
rects3 = ax.bar(ind+2*width, zvals, width, color='m')
rects4 = ax.bar(ind+3*width, kvals, width = width, color = 'g', 
                align='center', alpha=1, ecolor='black', capsize=8)


ax.set_title('Comparison of parameters', fontsize = 35)
ax.set_ylabel('Parameters', fontsize = 30)
ax.set_yticks([0, 0.5])
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(20) 
ax.set_xticks(ind+width)
ax.set_xticklabels( ('Appearance \n rate (box 1)', 'Appearance \n rate (box 2)','Grooming \n reward', 
                     'Traveling \n cost', 'Pushing button \n cost','policy \n temperature',
                    'Belief \n diffusion', 'direct prob'), rotation=60, 
                   ha = 'center', fontsize = 20)
leg = ax.legend( (rects1[0], rects2[0], rects3[0], rects4[0]), 
                ('init parameters 1', 'estimated parameters 1', 
                 'init parameters 2', 'estimated parameters 2' ), 
                fontsize = 10 , frameon=False)
leg_c = ['r', 'b', 'm', 'g']
for i, text in enumerate(leg.get_texts()):
    plt.setp(text, color = leg_c[i])

ax.xaxis.set_tick_params(length = 0)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
plt.show()

#fig_parameters.savefig('parameters.pdf', format='pdf', bbox_inches = 'tight')

# Behavioral statistics

In [ ]:
from twobox import *

In [ ]:
# nq = 10
# nr = 2
# nl = 3
# na = 5
# discount = 0.99

In [ ]:
T = len(obs)

In [ ]:
# point_est2_dis = np.insert(point_est2, 2, 0)
# point_est2_dis = np.insert(point_est2_dis, 2, 0)

In [ ]:
point_est2_dis = np.copy(point_est2)

In [ ]:
point_est2_dis

In [ ]:
# two_test2 = twoboxMDPdata(discount, nq, nr, na, nl, point_est2_dis, T, 1)

In [ ]:
# point_est2_dis

In [ ]:
# point_est3 = np.copy(point_est2)

In [ ]:
# point_est3

In [ ]:
# #temp = point_est3[0]
# #point_est3[0] = point_est3[1]
# #point_est3[1] = temp
# point_est3[0] *= 3
# point_est3[-2]/= 5
# point_est3[-3]/= 5

In [ ]:
# two_test3 = twoboxMDPdata(discount, nq, nr, na, nl, point_est3, T, 1)

In [ ]:
# two_test2.dataGenerate_sfm(0, 0, 0, 0)
# act_test2 = two_test2.action
# rew_test2 = two_test2.reward
# loc_test2 = two_test2.location


In [ ]:
twobox_est = twoboxMDP(discount, nq, nr, na, nl, point_est2_dis)
twobox_est.setupMDP()
twobox_est.solveMDP_sfm()
ThA = twobox_est.ThA
softpolicy = twobox_est.softpolicy
pi = np.ones(nq * nq)/ nq /nq  # initialize the estimation of the belief state
twoHMM_est = HMMtwobox(ThA, softpolicy, pi)

In [ ]:
# obs_test = (np.vstack([act_test2, rew_test2, loc_test2]).T)

In [ ]:
alpha_est, scale_est = twoHMM_est.forward_scale(obs)
beta_est = twoHMM_est.backward_scale(obs, scale_est)
gamma_est = twoHMM_est.compute_gamma(alpha_est, beta_est)
xi_est = twoHMM_est.compute_xi(alpha_est, beta_est, obs)

In [ ]:
belief1_est = np.sum(np.reshape(gamma_est[:, ].T, (gamma_est.shape[1], nq, nq)), axis = 2)
belief2_est = np.sum(np.reshape(gamma_est[:, ].T, (gamma_est.shape[1], nq, nq)), axis = 1)




In [ ]:
belief1_est = np.sum(np.reshape(alpha_est[:, ].T, (alpha_est.shape[1], nq, nq)), axis = 2)
belief2_est = np.sum(np.reshape(alpha_est[:, ].T, (alpha_est.shape[1], nq, nq)), axis = 1)


#fig_posterior = plt.figure(figsize= (15, 10))

showlen = 1000 #T // 1
startT = 700 #T // 1 * (1-1)

endT = startT + showlen
showT = range(startT,endT)

fig_posterior_monkey, [ax1, ax_loc, ax2] = plt.subplots(3, 1, figsize= (10, 6))
a = .5

ax1.imshow(1 - (belief1_est ** a)[showT].T, interpolation='Nearest', cmap='gray', origin='lower', aspect='auto')
ax1.yaxis.set_label_coords(-0.1,0.25)
ax1.set_ylabel('Belief posterior \n about box 1', rotation= 360, fontsize = 15)
ax1.set_yticks([0, nq-1])
ax1.set_yticklabels(['0','1'])
ax1.tick_params(axis='both', which='major', labelsize=18)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.set_xlim([0,showlen])

ax_loc.plot((np.remainder(loc[showT]+1, 3) - 1 ) * 10, 'g-o', markersize = 3)
box1_r = act[showT] // 4 * 7 * np.remainder(loc[showT]+1, 3) * np.insert(rew[showT][1:], -1, 0) * 1.0
box2_r = act[showT] // 4 * 7 * (np.remainder(loc[showT]+1, 3) - 2) * np.insert(rew[showT][1:], -1, 0) * 1.0
box1_n = act[showT] // 4 * 7 * np.remainder(loc[showT]+1, 3) * (1-np.insert(rew[showT][1:], -1, 0))* 1.0
box2_n = act[showT] // 4 * 7 * ((np.remainder(loc[showT]+1, 3) - 2) * (1-np.insert(rew[showT][1:], -1, 0)))* 1.0
box1_r[ box1_r==0 ] = np.nan
box2_n[ box2_n==0 ] = np.nan
box2_r[ box2_r==0 ] = np.nan
box1_n[ box1_n==0 ] = np.nan
ax_loc.plot(box2_r, '^', c = 'red', markersize = 10)
ax_loc.plot(box1_n, 'v', c = 'blue', markersize = 10)
ax_loc.plot(box2_n, '^', c = 'blue', markersize = 10)
ax_loc.plot(box1_r, 'v', c = 'red', markersize = 10)
ax_loc.set_xlim([0,showlen])
ax_loc.spines['top'].set_visible(False)
ax_loc.spines['right'].set_visible(False)
ax_loc.spines['bottom'].set_visible(False)
ax_loc.spines['left'].set_visible(False)
ax_loc.set_yticks([])

ax2.imshow(1 - (belief2_est ** a)[showT].T, interpolation='Nearest', cmap='gray', origin='lower',aspect='auto')
ax2.set_xlabel('time', fontsize = 18)
ax2.yaxis.set_label_coords(-0.1,0.25)
ax2.set_ylabel('Belief posterior \n about box 2', rotation= 360, fontsize = 15)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.set_yticks([0, nq-1])
ax2.set_yticklabels(['0','1'])
ax2.set_xlim([0,showlen])


plt.tight_layout()
plt.show()


#fig_posterior_monkey.savefig('fig_posterior_monkey.pdf', format='pdf', bbox_inches = 'tight')

In [ ]:
belief2_est[1200:1230]

In [ ]:
#fig_posterior = plt.figure(figsize= (15, 10))

showlen = 1000 #T // 1
startT = 700 #T // 1 * (1-1)

endT = startT + showlen
showT = range(startT,endT)

fig_posterior_monkey, [ax1, ax_loc, ax2] = plt.subplots(3, 1, figsize= (10, 6))
a = .5

ax1.imshow(1 - (belief1_est ** a)[showT].T, interpolation='Nearest', cmap='gray', origin='lower', aspect='auto')
ax1.yaxis.set_label_coords(-0.1,0.25)
ax1.set_ylabel('Belief posterior \n about box 1', rotation= 360, fontsize = 15)
ax1.set_yticks([0, nq-1])
ax1.set_yticklabels(['0','1'])
ax1.tick_params(axis='both', which='major', labelsize=18)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)
ax1.spines['left'].set_visible(False)
ax1.set_xlim([0,showlen])

ax_loc.plot((np.remainder(loc[showT]+1, 3) - 1 ) * 10, 'g-o', markersize = 3)
box1_r = act[showT] // 4 * 7 * np.remainder(loc[showT]+1, 3) * np.insert(rew[showT][1:], -1, 0) * 1.0
box2_r = act[showT] // 4 * 7 * (np.remainder(loc[showT]+1, 3) - 2) * np.insert(rew[showT][1:], -1, 0) * 1.0
box1_n = act[showT] // 4 * 7 * np.remainder(loc[showT]+1, 3) * (1-np.insert(rew[showT][1:], -1, 0))* 1.0
box2_n = act[showT] // 4 * 7 * ((np.remainder(loc[showT]+1, 3) - 2) * (1-np.insert(rew[showT][1:], -1, 0)))* 1.0
box1_r[ box1_r==0 ] = np.nan
box2_n[ box2_n==0 ] = np.nan
box2_r[ box2_r==0 ] = np.nan
box1_n[ box1_n==0 ] = np.nan
ax_loc.plot(box2_r, '^', c = 'red', markersize = 10)
ax_loc.plot(box1_n, 'v', c = 'blue', markersize = 10)
ax_loc.plot(box2_n, '^', c = 'blue', markersize = 10)
ax_loc.plot(box1_r, 'v', c = 'red', markersize = 10)
ax_loc.set_xlim([0,showlen])
ax_loc.spines['top'].set_visible(False)
ax_loc.spines['right'].set_visible(False)
ax_loc.spines['bottom'].set_visible(False)
ax_loc.spines['left'].set_visible(False)
ax_loc.set_yticks([])

ax2.imshow(1 - (belief2_est ** a)[showT].T, interpolation='Nearest', cmap='gray', origin='lower',aspect='auto')
ax2.set_xlabel('time', fontsize = 18)
ax2.yaxis.set_label_coords(-0.1,0.25)
ax2.set_ylabel('Belief posterior \n about box 2', rotation= 360, fontsize = 15)
ax2.tick_params(axis='both', which='major', labelsize=18)
ax2.set_yticks([0, nq-1])
ax2.set_yticklabels(['0','1'])
ax2.set_xlim([0,showlen])


plt.tight_layout()
plt.show()


#fig_posterior_monkey.savefig('fig_posterior_monkey.pdf', format='pdf', bbox_inches = 'tight')

In [ ]:
belief2_est[1200:1230]

In [ ]:
target_schedule

In [ ]:
[0.2 / point_est2[0], 0.2 / point_est2[1]]

In [ ]:
point_est2

In [ ]:
point_est_exp = np.copy(point_est2)
point_est_exp[0] = 1 / target_schedule[0] / 5
point_est_exp[1] = 1 / target_schedule[1] / 5

In [ ]:
point_est_exp[0] 

In [ ]:
twobox_est = twoboxMDP(discount, nq, nr, na, nl, point_est2)
twobox_est.setupMDP()
twobox_est.solveMDP_sfm()
ThA = twobox_est.ThA
softpolicy = twobox_est.softpolicy
pi = np.ones(nq * nq)/ nq /nq  # initialize the estimation of the belief state
twoHMM_est = HMMtwobox(ThA, softpolicy, pi)

log_likelihood_est = twoHMM_est.computeQaux(obs, ThA, softpolicy) + twoHMM_est.latent_entr(obs)

In [ ]:
log_likelihood_est

In [ ]:
twobox_est_exp = twoboxMDP(discount, nq, nr, na, nl, point_est_exp)
twobox_est_exp.setupMDP()
twobox_est_exp.solveMDP_sfm()
ThA_exp = twobox_est_exp.ThA
softpolicy_exp = twobox_est_exp.softpolicy
pi = np.ones(nq * nq)/ nq /nq  # initialize the estimation of the belief state
twoHMM_est_exp = HMMtwobox(ThA_exp, softpolicy_exp, pi)

log_likelihood_est_exp =twoHMM_est_exp.computeQaux(obs, ThA_exp, softpolicy_exp) + twoHMM_est_exp.latent_entr(obs)

In [ ]:
log_likelihood_est_exp

In [ ]:
N = 2
  
ind = np.arange(N)    
width = 0.3  
para_compar = np.array([log_likelihood_est, log_likelihood_est_exp])

fig_loglikelihood, _ = plt.subplots(figsize =(8, 8)) 
p1 = plt.bar(ind, para_compar, width) 


plt.ylabel('log-likelihood', fontsize = 15) 
plt.title('Comparison of log-likelihood', fontsize = 18) 
plt.xticks(ind, ('IRC parameter', 'IRC parameter with \n experiment setting'), fontsize = 15) 
plt.yticks([min(para_compar), 0], fontsize = 12) 

plt.show()
fig_loglikelihood.savefig('fig_loglikelihood.pdf', format='pdf', bbox_inches = 'tight')

In [ ]:
para_compar

In [ ]:
plt.imshow(belief1_est[500:600, :].T)
plt.show()

In [ ]:
plt.hist(np.squeeze(loc_test3))
plt.show()

In [ ]:
print(len(np.where((rew_test3 == 1) & (loc_test3 == 1))[1]), len(np.where((rew_test3 == 1) & (loc_test3 == 2))[1]))

In [ ]:
point_est3

In [ ]:
two_test.dataGenerate_sfm(0, 0, 0, 0)

In [ ]:
act_test = two_test.action
rew_test = two_test.reward
loc_test = two_test.location

In [ ]:
plt.plot(np.squeeze(loc_test))
plt.show()

In [ ]:
plt.hist(np.squeeze(loc_test))
plt.show()

In [ ]:
len(np.where((rew_test == 1) & (loc_test == 2))[1])

In [ ]:
point_est2

In [ ]:
pbind_test = np.where(np.squeeze(act_test) == 4)[0]
pbind = np.where(np.squeeze(act) == 4)[0]
# # time between two pb



In [ ]:
travelind_test = np.concatenate((np.where(np.squeeze(act_test) == 1)[0], 
                              np.where(np.squeeze(act_test) == 2)[0],
                              np.where(np.squeeze(act_test) == 3)[0]))
travelind = np.concatenate((np.where(np.squeeze(act) == 1)[0], 
                              np.where(np.squeeze(act) == 2)[0],
                              np.where(np.squeeze(act) == 3)[0]))



In [ ]:
len(act)

In [ ]:
np.squeeze(act_test)

In [ ]:
fig_behav, ax = plt.subplots(2, 2, figsize=(8, 8))

ax[0, 0].hist([np.squeeze(act), np.squeeze(act_test)], bins = [0,1,2,3,4,5], 
         color = ['red','blue'], label = ['monkey','IRCagent'], alpha = 0.8)
ax[0, 0].legend(fontsize = 12, frameon = False)
ax[0, 0].set_title('Actions', fontsize = 30)
labels = [item.get_text() for item in ax[0, 0].get_xticklabels()]
labels[1] = 'do \n nothing'
labels[2] = 'go to \n center'
labels[3] = 'go to 1'
labels[4] = 'go to 2'
labels[5] = 'push \n button'
ax[0, 0].set_ylabel('Frequency', fontsize = 20)
ax[0, 0].set_xticklabels(labels, rotation=60, ha = 'center', fontsize = 20)
ax[0, 0].xaxis.set_tick_params(length = 0)
ax[0, 0].set_yticks([])
ax[0, 0].spines['top'].set_visible(False)
ax[0, 0].spines['right'].set_visible(False)
ax[0, 0].spines['bottom'].set_visible(False)
ax[0, 0].spines['left'].set_visible(False)

weights_loc = np.ones_like(np.squeeze(loc))/float(len(np.squeeze(loc)))
weights_loc_test = np.ones_like(np.squeeze(loc_test))/float(len(np.squeeze(loc_test)))
ax[0, 1].set_title('Locations', fontsize = 30)
ax[0, 1].hist([np.squeeze(loc), np.squeeze(loc_test)], weights = [weights_loc, weights_loc_test],
         bins = 5, color = ['red','blue'], label = ['monkey', 'IRCagent'], alpha = 0.8)
ax[0, 1].set_xticks(np.arange(0, 2, 0.2))
labels = [item.get_text() for item in ax[0, 1].get_xticklabels()]
labels[1] = 'center'
labels[5] = 'box 1'
labels[9] = 'box 2'
ax[0, 1].set_xticklabels(labels, ha = 'center', fontsize = 20)
ax[0, 1].xaxis.set_tick_params(length = 0)
ax[0, 1].set_yticks([])
ax[0, 1].set_ylabel('Frequency', fontsize = 20)
ax[0, 1].spines['top'].set_visible(False)
ax[0, 1].spines['right'].set_visible(False)
ax[0, 1].spines['bottom'].set_visible(False)
ax[0, 1].spines['left'].set_visible(False)


weights_pbind = np.ones_like(pbind[1:])/float(len(pbind[1:] ))
weights_pbind_test = np.ones_like(pbind_test[1:])/float(len(pbind_test[1:]))
ax[1, 0].set_title('Time between \n button pushes', fontsize = 30)
ax[1, 0].hist([pbind[1:] - pbind[0:-1], 
               pbind_test[1:] - pbind_test[0:-1]],
              weights = [weights_pbind, weights_pbind_test],
              bins = np.linspace(0, 40, 8), alpha = 0.8, 
              color = ['red','blue'], label = ['monkey', 'IRCagent'])
ax[1, 0].set_ylabel('Frequency', fontsize = 20)
ax[1, 0].set_xticks(range(0,41, 3))
labels = [item.get_text() for item in ax[1, 0].get_xticklabels()]
labels[1] = '0-5'
labels[6] = '15-20 \n time'
labels[11] = '35-40'
ax[1, 0].set_xticklabels(labels, ha = 'center', fontsize = 20)
ax[1, 0].xaxis.set_tick_params(length = 0)
ax[1, 0].set_yticks([])
ax[1, 0].spines['top'].set_visible(False)
ax[1, 0].spines['right'].set_visible(False)
ax[1, 0].spines['bottom'].set_visible(False)
ax[1, 0].spines['left'].set_visible(False)


weights_travelind = np.ones_like(travelind[1:])/float(len(travelind[1:] ))
weights_travelind_test = np.ones_like(travelind_test[1:])/float(len(travelind_test[1:]))
ax[1, 1].set_title('Time between \n travel', fontsize = 30)
ax[1, 1].set_ylabel('Frequency', fontsize = 20)
ax[1, 1].hist([travelind[1:] - travelind[0:-1], 
               travelind_test[1:] - travelind_test[0:-1]], 
              weights = [weights_travelind, weights_travelind_test], 
         bins = np.linspace(0, 40, 8), 
         color = ['red','blue'], label = ['monkey', 'IRCagent'], alpha = 0.8)
ax[1, 1].set_xticks(range(0,41, 3))
labels = [item.get_text() for item in ax[1, 1].get_xticklabels()]
labels[1] = '0-5'
labels[6] = '15-20 \n time'
labels[11] = '35-40'
ax[1, 1].set_xticklabels(labels, ha = 'center', fontsize = 20)
ax[1, 1].xaxis.set_tick_params(length = 0)
ax[1, 1].set_yticks([])
ax[1, 1].spines['top'].set_visible(False)
ax[1, 1].spines['right'].set_visible(False)
ax[1, 1].spines['bottom'].set_visible(False)
ax[1, 1].spines['left'].set_visible(False)




plt.tight_layout()
plt.show()
#fig_behav.savefig('behavior_state.pdf', format='pdf', bbox_inches = 'tight')


